# Ejercicio Block Cipher
Diego Andrés Morales Aquino
21762


## 1. Generación una función cifrado y descifrado DES 

Código realizado con el apoyo del modelo GPT 3.5 <br>
Prompt disponible en: https://chatgpt.com/share/67ca43b1-7c64-8009-b4f7-8467992c94ff

In [47]:
from Crypto.Cipher import DES
from Crypto.Random import get_random_bytes
import binascii

In [48]:

def add_padding(text):
    """
    Agregar padding. El padding está compuesto por el char que corresponde a la longitud del padding.
    """
    pad_length = 8 - len(text) % 8
    padding = chr(pad_length) * pad_length
    return text + padding

def des_encrypt(plain_text):

    key = get_random_bytes(8)
    padded_text = add_padding(plain_text)
    
    # Cifrado DES en modo ECB
    cipher = DES.new(key, DES.MODE_ECB)
    encrypted_text = cipher.encrypt(padded_text.encode())
    
    # Retornar llave y cifrado en hexa
    return binascii.hexlify(key).decode(), binascii.hexlify(encrypted_text).decode()

In [49]:
def remove_padding(padded_text):
    """
    Obtener la longitud del padding a partir del ascii del último char y eliminarlo.
    """
    pad_length = ord(padded_text[-1])
    return padded_text[:-pad_length]

def des_decrypt(key_hex, encrypted_text_hex):

    # hex a bytes
    key = binascii.unhexlify(key_hex)
    encrypted_text = binascii.unhexlify(encrypted_text_hex)
    
    # Desencriptar con DES
    cipher = DES.new(key, DES.MODE_ECB)
    decrypted_text = cipher.decrypt(encrypted_text)
    
    # Eliminar el padding
    original_text = remove_padding(decrypted_text.decode())
    
    return original_text

In [50]:
# Encriptar texto
plain_text = "Mensaje de Diego Morales."
key, encrypted_text = des_encrypt(plain_text)

print(f"Llave generada: {key}")
print(f"Texto cifrado: {encrypted_text}")

Llave generada: a2631d625148c532
Texto cifrado: 6809d205127b071b967bf35d8aa69ab12690ff3c50914b9f507340e8d4b391aa


In [51]:
# Desencriptar el texto
original_text = des_decrypt(key, encrypted_text)

print(f"Texto original: {original_text}")

Texto original: Mensaje de Diego Morales.


## 2. Generación una función cifrado y descifrado 3DES

In [52]:
from Crypto.Cipher import DES3
from Crypto.Util.Padding import pad, unpad
from Crypto.Random import get_random_bytes
import base64

In [53]:
def encrypt_3des_cbc(message: str) -> str:
    # Clave de 24 bytes
    key = get_random_bytes(24)
    
    # Generar un vector de inicialización
    iv = get_random_bytes(8)
    
    cipher = DES3.new(key, DES3.MODE_CBC, iv)
    
    # Convertir a bytes y hacer padding hasta que sea múltiplo del tamaño de bloque
    message_bytes = message.encode('utf-8')
    padded_message = pad(message_bytes, DES3.block_size)
    
    # Cifrar
    encrypted_bytes = cipher.encrypt(padded_message)
    
    # Retornar la clave, IV y el mensaje cifrado en base64
    return base64.b64encode(key + iv + encrypted_bytes).decode('utf-8')

In [54]:
def decrypt_3des_cbc(encrypted_message: str) -> str:
    # Decodificar desde base64
    encrypted_data = base64.b64decode(encrypted_message)
    
    # Viene en formato key (24b)+iv(8b)+encrypted_message
    key = encrypted_data[:24]
    iv = encrypted_data[24:32]
    encrypted_bytes = encrypted_data[32:]
    
    cipher = DES3.new(key, DES3.MODE_CBC, iv)
    
    # Desencriptar y quitar el padding
    decrypted_padded = cipher.decrypt(encrypted_bytes)
    decrypted_message = unpad(decrypted_padded, DES3.block_size)
    
    return decrypted_message.decode('utf-8')

In [55]:
print("Mensaje crifrado con 3DES:")
mensaje_cifrado = encrypt_3des_cbc("contraseña: 123456")
print(mensaje_cifrado)

Mensaje crifrado con 3DES:
//wTSLiMU8rTITl0PrzD3ra2PsoBNhop2RpxJYRojcTNarYXGJ4DtBQIF19M8z0foVqyh5PNFjk=


In [56]:
print("Mensaje descifrado con 3DES:")
mensaje_descifrado = decrypt_3des_cbc(mensaje_cifrado)
print(mensaje_descifrado)


Mensaje descifrado con 3DES:
contraseña: 123456


## 3. Generación una función cifrado y descifrado AES con CBC Y ECB

Realizado con el apoyo de GPT 3.5 https://chatgpt.com/share/67d39e0c-009c-8009-b6b1-c70f0d23dcc6

In [57]:
from PIL import Image
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
from Crypto.Random import get_random_bytes
import numpy as np
import os

In [58]:
def encrypt_image_aes(img_path, mode="ECB"):
    # Cargar imagen y convertir a RGB
    img = Image.open(img_path).convert("RGB")
    img_array = np.array(img)
    img_bytes = img_array.tobytes()
    
    # Generar key
    key = get_random_bytes(16)
    
    if mode == "CBC":
        iv = get_random_bytes(16)  # Generar vecotr de inicialización
        cipher = AES.new(key, AES.MODE_CBC, iv)
    else:
        cipher = AES.new(key, AES.MODE_ECB)
        iv = None  # No se usa vector de inicialización en ECB
    
    # Añadir padding y cifrar
    padded_bytes = pad(img_bytes, AES.block_size)
    encrypted_bytes = cipher.encrypt(padded_bytes)
    
    # Guardar como imagen
    encrypted_array = np.frombuffer(encrypted_bytes, dtype=np.uint8)
    encrypted_array = encrypted_array[:img_array.size].reshape(img_array.shape)  
    encrypted_img = Image.fromarray(encrypted_array)
    
    output_file = f"encrypted_{mode}_{os.path.basename(img_path)}"
    encrypted_img.save(output_file)
    
    return key, iv, img.size, output_file

In [59]:
def decrypt_image_aes(encrypted_png_path, key, iv=None, mode="ECB", img_size=(0, 0)):
    # Cargar imagen cifrada
    encrypted_img = Image.open(encrypted_png_path).convert("RGB")
    encrypted_array = np.array(encrypted_img)
    encrypted_bytes = encrypted_array.tobytes()
    
    # Añadir padding si no es múltiplo del tamaño de bloque
    block_size = AES.block_size
    if len(encrypted_bytes) % block_size != 0:
        padding_length = block_size - (len(encrypted_bytes) % block_size)
        encrypted_bytes += b'\0' * padding_length
    
    # crear objeto para cifrar según el modo
    if mode == "CBC":
        cipher = AES.new(key, AES.MODE_CBC, iv)
    else:
        cipher = AES.new(key, AES.MODE_ECB)
    
    try:
        # Descifrar y eliminar padding
        decrypted_bytes = unpad(cipher.decrypt(encrypted_bytes), AES.block_size)
    except ValueError:
        # Si hay un error con el padding, intentar sin desempaquetar
        decrypted_bytes = cipher.decrypt(encrypted_bytes)
        # Limitar bytes al tamaño original
        decrypted_bytes = decrypted_bytes[:img_size[0] * img_size[1] * 3]
    
    # Convertir a imagen
    decrypted_array = np.frombuffer(decrypted_bytes, dtype=np.uint8)
    decrypted_array = decrypted_array[:img_size[0] * img_size[1] * 3].reshape((img_size[1], img_size[0], 3))
    decrypted_img = Image.fromarray(decrypted_array)
    
    return decrypted_img

In [60]:
# Encriptar imagen con ECB
img_path = "pic.png"
print("Cifrando en modo ECB...")
key_ecb, _, img_size, ecb_output = encrypt_image_aes(img_path, mode="ECB")
print(f"Imagen cifrada guardada como: {ecb_output}")

Cifrando en modo ECB...
Imagen cifrada guardada como: encrypted_ECB_pic.png


In [61]:
# Descifrar imagen con ECB
print("\nDescifrando imagen ECB...")
decrypted_ecb_img = decrypt_image_aes(ecb_output, key_ecb, mode="ECB", img_size=img_size)
decrypted_ecb_output = "decrypted_ECB.png"
decrypted_ecb_img.save(decrypted_ecb_output)
print(f"Imagen descifrada guardada como: {decrypted_ecb_output}")


Descifrando imagen ECB...
Imagen descifrada guardada como: decrypted_ECB.png


In [62]:
# Encriptar imagen con CBC
print("\nCifrando en modo CBC...")
key_cbc, iv_cbc, img_size, cbc_output = encrypt_image_aes(img_path, mode="CBC")
print(f"Imagen cifrada guardada como: {cbc_output}")


Cifrando en modo CBC...


Imagen cifrada guardada como: encrypted_CBC_pic.png


In [63]:
# Descifrar imagen con CBC
print("\nDescifrando imagen CBC...")
decrypted_cbc_img = decrypt_image_aes(cbc_output, key_cbc, iv_cbc, mode="CBC", img_size=img_size)
decrypted_cbc_output = "decrypted_CBC.png"
decrypted_cbc_img.save(decrypted_cbc_output)
print(f"Imagen descifrada guardada como: {decrypted_cbc_output}")


Descifrando imagen CBC...
Imagen descifrada guardada como: decrypted_CBC.png


## Pruebas unitarias
Generadas con el modelo Claude. Conversación disponible en: https://claude.ai/share/0bbb24cc-fa60-4e3c-a2ca-f3368326f617

In [64]:
def assert_true(condition, message):
    if not condition:
        raise AssertionError(message)

def test_des():
    
    # Caso básico: texto simple
    plain_text = "Mensaje de prueba"
    key, encrypted_text = des_encrypt(plain_text)
    
    # Descifrar y verificar
    decrypted_text = des_decrypt(key, encrypted_text)
    if decrypted_text != plain_text:
        raise AssertionError(f"Error en DES: texto descifrado '{decrypted_text}' no coincide con original '{plain_text}'")
    
    print("Prueba DES completada")

def test_3des():
    
    # Caso básico
    plain_text = "contraseña secreta"
    encrypted = encrypt_3des_cbc(plain_text)
    
    # Descifrar y verificar
    decrypted = decrypt_3des_cbc(encrypted)
    if decrypted != plain_text:
        raise AssertionError(f"Error en 3DES: texto descifrado '{decrypted}' no coincide con original '{plain_text}'")
    
    print("Prueba 3DES completada")

def test_aes_ecb():
    
    # Crear una imagen de prueba simple
    test_image_path = "test_image.png"
    test_img = Image.new('RGB', (20, 20), color=(100, 150, 200))
    test_img.save(test_image_path)
    
    # Cifrar imagen en modo ECB
    try:
        key_ecb, _, img_size, ecb_output = encrypt_image_aes(test_image_path, mode="ECB")
        
        # Verificar archivo creado
        if not os.path.exists(ecb_output):
            raise AssertionError("No se creó el archivo de imagen cifrada")
        
        # Descifrar
        decrypt_image_aes(ecb_output, key_ecb, mode="ECB", img_size=img_size)
        
        # Limpiar archivos
        os.remove(test_image_path)
        os.remove(ecb_output)
        
        print("Prueba AES-ECB completada")
    except Exception as e:
        # Limpiar archivos en caso de error
        if os.path.exists(test_image_path):
            os.remove(test_image_path)
        if os.path.exists(ecb_output):
            os.remove(ecb_output)
        raise AssertionError(f"Error en AES-ECB: {str(e)}")
    
def test_aes_cbc():
    
    test_image_path = "test_image.png"
    
    # Verificar que la imagen de prueba existe
    if not os.path.exists(test_image_path):
        test_img = Image.new('RGB', (20, 20), color=(100, 150, 200))
        test_img.save(test_image_path)
    
    # Cifrar imagen en modo CBC
    try:
        key_cbc, iv_cbc, img_size, cbc_output = encrypt_image_aes(test_image_path, mode="CBC")
        
        # Verificar archivo creado
        if not os.path.exists(cbc_output):
            raise AssertionError("No se creó el archivo de imagen cifrada en modo CBC")
        
        # Descifrar
        decrypt_image_aes(cbc_output, key_cbc, iv_cbc, mode="CBC", img_size=img_size)
        
        # Limpiar archivos
        os.remove(cbc_output)
        os.remove(test_image_path)  # Eliminar la imagen de prueba al final
        
        print("Prueba AES-CBC completada")
    except Exception as e:
        # Limpiar archivos en caso de error
        if os.path.exists(cbc_output):
            os.remove(cbc_output)
        if os.path.exists(test_image_path):
            os.remove(test_image_path)
        raise AssertionError(f"Error en AES-CBC: {str(e)}")


def run_tests():
    print("Ejecutando pruebas unitarias simplificadas...\n")
    
    try:
        # Ejecutar pruebas
        test_des()
        test_3des()
        test_aes_ecb()
        test_aes_cbc()
        
        print("\nTodas las pruebas pasaron correctamente!")
    except AssertionError as e:
        print(f"\nError: {str(e)}")
    except Exception as e:
        print(f"\nError inesperado: {str(e)}")



run_tests()

Ejecutando pruebas unitarias simplificadas...

Prueba DES completada
Prueba 3DES completada
Prueba AES-ECB completada
Prueba AES-CBC completada

Todas las pruebas pasaron correctamente!


## 4. Preguntas a Responder 

- **¿Qué tamaño de clave se está usando para DES, 3DES y AES?** <br>

    Para DES se utiliza un tamaño de llave de 8 bytes, en donde 56 bits corresponden a la llave como tal y los 8 bits restantes se utilizan para verificaciones de paridad. Para 3DES se emplea una llave de 24 bytes, que se subdivide en 3 llaves distintas de 8 bytes cada una, para aplicar DES 3 veces. Y por último, para AES se está utilizando una llave de 16 bytes o 128 bits. 

- **¿Qué modo de operación está implementado?** <br>

    ECB y CBC.

- **¿Por qué no debemos usar ECB en datos sensibles?** <br>

    Debido a que ECB cifra cada bloque de forma independiente, por lo que podría darse el caso en el que dos o más bloques son idénticos, resultando en el mismo cifrado. Esto a su vez permite ver patrones visibles en los datos cifrados, tal y como se observa en la imagen cifrada con AES CBC, en donde se pueden identificar rasgos de la imagen origianl.

- **¿Cual es la diferencia entre ECB vs CBC, se puede notar directamente en una imagen?** <br>

    El modo ECB cifra cada bloque de forma independiente, por lo que se pueden generar cifrados identicos para bloques de entrada iguales. Mientras que, CBC cifra cada bloque dependiendo del bloque anterior y de un vector de inicialización (en el caso del primero). En la imagen cifrada con ECB se pueden observar patrones, tal y como en la imagen de ejemplo que es posible detectar razgos de la imagen original. Mientras que CBC al introducir un mayor grado de aleatoriedad, permite ocultar de mejor manera patrones, dando como resultado una imagen cifrada que a simple vista parece únicamente ruido.

- **¿Que es el IV?** <br>

    El IV o vector de inicialización corresponde a un valor aleatorio o único que se utiliza en modos de cifrado como CBC para agregar aleatoriedad y evitar que al cifrar un mismo bloque se obtenga dos veces el mismo resultado. Este vector debe tener el mismo tamaño de bloque, no es necesario que se mantenga en secreto y no debe reutilizarse para cifrar con la misma clave. Por ejemplo en CBC, el primer bloque de texto claro se combina con el IV mediante una operación XOR antes de ser cifrado. Esto asegura que incluso se cifra el mismo texto con la misma llave el resultado  será distinto, debido al elemento aleatorio que introduce el IV al inicio de la cadena de cifrado.

- **¿Que es el PADDING?** <br>

    El padding corresponde a los bits que se requieren para que el tamaño del texto a cifrar sea múltiplo del tamaño de bloque que el algoritmo de cifrado requiere. Es decir, el padding consiste en datos adicionales triviales que se tienen que agregar para completar el último bloque.

- **¿En qué situaciones se recomienda cada modo de operación?** <br>

    ECB puede ser útil para entornos donde se requiera rapidez y no se cuente con datos sensibles, ya que revela patrones en los datos cifrados. CBC es más seguro para datos sensibles, pero puede ser más lento por su cifrado secuencial. CTR es más rápido y permite paralelización, lo que lo hace adecuado para situaciones que requieren alto rendimiento sin comprometer la seguridad. Por otro lado se cuenta con los modos OFB, que es ideal para cifrado en tiempo real y GCM para aplicaciones en donde es importante la protección contra la modificación y la confidencialidad de los datos, dado que permite autenticar los datos además de cifrarlo. 

- **¿Cómo elegir un modo seguro en cada lenguaje de programación?** <br>

    En python, para elegir el modo seguro CBC, se realiza de la siguiente manera:

    - DES
        ```
        cipher = DES.new(key, DES.MODE_ECB)
        ```
    - 3DES
        ```
        cipher = DES3.new(key, DES3.MODE_CBC, iv)
        ```

    - AES
        ```
        cipher = Cipher(algorithms.AES(key), modes.CBC(iv))
        ```